# Report: AIS Vessel Trajectory Prediction

## Exploratory Data Analysis

### Domain knowledge
AIS is a short range tracking system that transmits ships positions using the high frequency maritime frequency band. It was originally developed as a collision avoidance tool. Each ship transmits a signal containing vessel identity, position, speed and course to surrounding ships and shore stations. The position of the ship is obtained by either using the ships GPS signal or an inertial sensor built into the AIS unit. Static information is sent every sixth minute or at request the ship sends MMSI number, IMO number, name and call sign, length and beam, type of ship and location of position fixing antenna. Dynamic information is sent dependent on the speed and course alterations of the ship. The dynamic information consist of Ships position, accuracy indication, position timestamp and the course over ground. 

It is mandatory for all ships above 300 GT (gross tonnage) to carry AIS, and it is illegal to turn it of. All 300 GT boats mus carry a class A AIS system, while other boats like pleasure crafts can carry a class B AIS system with limited functionality. 

The signals are sensible to noise and enviromental disturbances. Making AIS unpredictable. Some relevant factors are signal propagation conditions, sea state, the height of antennas and strength of the transmitter. These factors giva a large variation in signal transmition length from 20-350 nautical miles. But an average of 40 nautical miles can safely be expected.

In [25]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from datetime import timedelta


### Data intuitition

Heading is given in angular degrees from north from 0 deg to 359 deg. It is also quite noisy and could use filtering first. 

In [26]:
def moving_average(data, window_size):
    return np.convolve(data, np.ones(window_size) / window_size, mode='same')

# Example: Applying a moving average filter
window_size = 5  # You can adjust the window size based on the level of smoothing you need

def exponential_moving_average(data, alpha):
    ema = [data[0]]  # Initialize with the first data point
    for i in range(1, len(data)):
        ema.append(alpha * data[i] + (1 - alpha) * ema[i - 1])
    return np.array(ema)

# Example: Applying EMA
alpha = 0.3  # Smoothing factor (0 < alpha <= 1)

def low_pass_filter(data, cutoff_freq, sampling_rate):
    fft_data = np.fft.fft(data)
    frequencies = np.fft.fftfreq(len(data), d=1/sampling_rate)
    
    # Zero out frequencies higher than the cutoff
    fft_data[np.abs(frequencies) > cutoff_freq] = 0
    
    # Inverse FFT to get the filtered signal
    return np.fft.ifft(fft_data)

# Example: Applying a low-pass filter
cutoff_frequency = 0.1  # Define the cutoff frequency
sampling_rate = 1  # Define the sampling rate (based on time intervals of data)

There is a spread in how many samples there are in each timeslot. The AIS should be periodically and it should have the same number within each timeslot. There is also a big difference in the number of samples between the different ships. There is also huge gaps between each sequence of AIS data. Could split up each ship in continous slots where AIS is available. Seems like it spams AIS data when they are stationary. 

The cog measurement is really noisy.



### Data generation

## Predictors

In [36]:
sched_df = pd.read_csv('../datasets/schedules_to_may_2024.csv', sep='|')
sched_df['arrivalDate'] = pd.to_datetime(sched_df['arrivalDate'])
sched_df['sailingDate'] = pd.to_datetime(sched_df['sailingDate'])
sched_df = sched_df.sort_values(by=['vesselId', 'arrivalDate'])

def get_next_dest_and_time_left(row, prev_pos_avail=True):
    vesselId, timestamp = row['vesselId'], row['time']
    dest_lon = None
    dest_lat = None
    time_left = None
    for idx, r in sched_df[sched_df['vesselId'] == vesselId].iterrows():
        if timestamp < r['arrivalDate'].replace(tzinfo=None):
            dest_lon = r['portLongitude']
            dest_lat = r['portLatitude']
            time_left = r['arrivalDate'].replace(tzinfo=None) - timestamp
            ret_is_valid = int(not np.isnan(dest_lon) and not np.isnan(dest_lat))
            if not ret_is_valid:
                if not prev_pos_avail:
                    lon = row['longitude']
                    lat = row['latitude']
                else:
                    lon = row['prev_lon']
                    lat = row['prev_lat']
                return lon, lat, 1, ret_is_valid
            return dest_lon, dest_lat, time_left.total_seconds(), ret_is_valid
    if not prev_pos_avail:
        lon = row['longitude']
        lat = row['latitude']
    else:
        lon = row['prev_lon']
        lat = row['prev_lat']
    return lon, lat, 1, 0

def should_be_moored(row):
    vesselId, timestamp = row['vesselId'], row['time']
    for idx, r in sched_df[(sched_df['vesselId'] == vesselId) & (sched_df['arrivalDate'] < timestamp.replace(tzinfo=timezone.utc)) & (sched_df['arrivalDate'] > datetime(2023, 12, 1, 0, 0, tzinfo=timezone.utc))].iterrows():
        if timestamp > r['arrivalDate'].replace(tzinfo=None):
            if timestamp < r['sailingDate'].replace(tzinfo=None):
                return 1 
        else:
            return 0
    return 0

In [37]:
filepath_train = '../datasets/ais_train.csv'
filepath_test = '../datasets/ais_test.csv'

# Load AIS historical data
train = pd.read_csv(filepath_train, sep ='|')  # Replace with your dataset
test = pd.read_csv(filepath_test, sep = ',')

# Preprocessing
train['time'] = pd.to_datetime(train['time'])
train.sort_values(by=['vesselId', 'time'], inplace=True)
train['isMoored'] = train['navstat']== 5
# Why are we using the complement?
train = train[~train['isMoored']]

test['time'] = pd.to_datetime(test['time'])
test.sort_values(by=['vesselId', 'time'], inplace=True)

# Feature Engineering
train['prev_lat'] = train.groupby('vesselId')['latitude'].shift(1)
train['prev_lon'] = train.groupby('vesselId')['longitude'].shift(1)
train['prev_speed'] = train.groupby('vesselId')['sog'].shift(1)
train['prev_course'] = (train.groupby('vesselId')['cog'].shift(1) / 180) - 1        # normalized
train['prev_rotation'] = train.groupby('vesselId')['rot'].shift(1) 
train['prev_heading'] = (train.groupby('vesselId')['heading'].shift(1)/ 180) - 1 
train['hour'] = train['time'].dt.hour
# Could change this to is_weekend
train['day_of_week'] = train['time'].dt.dayofweek
# Adding timedelta as a feature
train['time_diff'] = train['time'].diff()
train['time_diff_seconds'] = train['time_diff'].dt.total_seconds()


# --------------------------------- prev_rot-related stuff
# Replace special values with NaN
train['prev_rotation'] = train['prev_rotation'].replace({127: np.nan, -127: np.nan, -128: np.nan})
# Optional: Create a new column for turn information availability
train['turn_info_available'] = np.where(train['prev_rotation'] == -128, 0, 1)

# Create binary columns for turn direction and magnitude
train['turn_direction'] = np.where(train['prev_rotation'] > 0, 'right', 'left')
train['turn_magnitude'] = train['prev_rotation'].abs()

# Fill missing values (optional, using forward fill) Uses most recent non-null value from the row above.
train['prev_rotation'].fillna(method='ffill', inplace=True)

# Drop rows with missing values
train.dropna(inplace=True)

print(f"Length of dataset after preprocessing: {len(train)}")

# Define features and target variables
X = train[['prev_lat', 'prev_lon', 'prev_speed', 'prev_course','prev_rotation', 'turn_info_available', 'prev_heading', 'time_diff_seconds']]
y_lat = train['latitude']
y_lon = train['longitude']
X_test = test

X_lat_train, X_lat_val, y_lat_train, y_lat_val = train_test_split(X, y_lat, test_size=0.1, random_state=42)
X_lon_train, X_lon_val, y_lon_train, y_lon_val = train_test_split(X, y_lon, test_size=0.1, random_state=42)

# Train the model
model_lat = RandomForestRegressor(n_estimators=20, verbose=3, random_state=42)
model_lat.fit(X_lat_train.values, y_lat_train.values)

model_lon = RandomForestRegressor(n_estimators=20, verbose=3, random_state=42)
model_lon.fit(X_lon_train.values, y_lon_train.values)

# Make predictions on the validation set
y_lat_pred_val = model_lat.predict(X_lat_val)
y_lon_pred_val = model_lon.predict(X_lon_val)

# Evaluate performance on the validation set
mae_lat = mean_absolute_error(y_lat_val, y_lat_pred_val)
mae_lon = mean_absolute_error(y_lon_val, y_lon_pred_val)

print(f'Mean Absolute Error for Latitude: {mae_lat}')
print(f'Mean Absolute Error for Longitude: {mae_lon}')


C:\Users\marvaar\AppData\Local\Temp\ipykernel_15180\779510814.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['prev_rotation'].fillna(method='ffill', inplace=True)
C:\Users\marvaar\AppData\Local\Temp\ipykernel_15180\779510814.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['prev_rotation'].fillna(method='ffill', inplace=True)


KeyboardInterrupt: 

In [45]:
filepath_train = '../datasets/ais_train_preprocessed.csv'
filepath_test = '../datasets/ais_test.csv'

# Load AIS historical data
train = pd.read_csv(filepath_train, sep =',')  # Replace with your dataset

# Preprocessing
print(train.columns)
train['time'] = pd.to_datetime(train['time'])
train.sort_values(by=['vesselId', 'time'], inplace=True)
train['isMoored'] = train['navstat']== 5
# Why are we using the complement?
train = train[~train['isMoored']]

print(f"Length of training dataset: {len(train)}")

# Define features and target variables
X = train[['prev_lat', 'prev_lon', 'prev_speed', 'prev_course','prev_rotation', 'turn_info_available', 'prev_heading', 'time_diff_seconds', 'should_be_moored', 'dest_lon', 'dest_lat', 'time_left', 'sched_data_available', 'day_of_week']]
y_lat = train['latitude']
y_lon = train['longitude']

X_lat_train, X_lat_val, y_lat_train, y_lat_val = train_test_split(X, y_lat, test_size=0.1, random_state=42)
X_lon_train, X_lon_val, y_lon_train, y_lon_val = train_test_split(X, y_lon, test_size=0.1, random_state=42)

# Train the model
model_lat = RandomForestRegressor(n_estimators=20, verbose=3, random_state=42)
model_lat.fit(X_lat_train.values, y_lat_train.values)

model_lon = RandomForestRegressor(n_estimators=20, verbose=3, random_state=42)
model_lon.fit(X_lon_train.values, y_lon_train.values)

# Make predictions on the validation set
y_lat_pred_val = model_lat.predict(X_lat_val)
y_lon_pred_val = model_lon.predict(X_lon_val)

# Evaluate performance on the validation set
mae_lat = mean_absolute_error(y_lat_val, y_lat_pred_val)
mae_lon = mean_absolute_error(y_lon_val, y_lon_pred_val)

print(f'Mean Absolute Error for Latitude: {mae_lat}')
print(f'Mean Absolute Error for Longitude: {mae_lon}')


Index(['Unnamed: 0', 'time', 'cog', 'sog', 'rot', 'heading', 'navstat',
       'etaRaw', 'latitude', 'longitude', 'vesselId', 'portId', 'prev_lat',
       'prev_lon', 'prev_speed', 'prev_course', 'prev_rotation',
       'prev_heading', 'hour', 'day_of_week', 'time_diff', 'time_diff_seconds',
       'turn_info_available', 'turn_direction', 'turn_magnitude',
       'should_be_moored', 'dest_lon', 'dest_lat', 'time_left',
       'sched_data_available'],
      dtype='object')
Length of training dataset: 894129
building tree 1 of 20
building tree 2 of 20
building tree 3 of 20
building tree 4 of 20
building tree 5 of 20
building tree 6 of 20
building tree 7 of 20
building tree 8 of 20
building tree 9 of 20
building tree 10 of 20
building tree 11 of 20
building tree 12 of 20
building tree 13 of 20
building tree 14 of 20
building tree 15 of 20
building tree 16 of 20
building tree 17 of 20
building tree 18 of 20
building tree 19 of 20
building tree 20 of 20
building tree 1 of 20
building tree 2

c:\Users\marvaar\Documents\tdt4173\venv_tdt4173\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
c:\Users\marvaar\Documents\tdt4173\venv_tdt4173\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


Mean Absolute Error for Latitude: 0.07214914520212934
Mean Absolute Error for Longitude: 0.11517558508570855


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from datetime import timedelta


In [2]:
def make_training_set(n_shifts):
    filepath_train = '../datasets/ais_train.csv'
    filepath_test = '../datasets/ais_test.csv'

    # Load AIS historical data
    train = pd.read_csv(filepath_train, sep ='|')  # Replace with your dataset
    test = pd.read_csv(filepath_test, sep = ',')

    # Preprocessing
    train['time'] = pd.to_datetime(train['time'])
    train.sort_values(by=['vesselId', 'time'], inplace=True)
    train['isMoored'] = train['navstat']== 5
    # Why are we using the complement?
    train = train[~train['isMoored']]

    test['time'] = pd.to_datetime(test['time'])
    test.sort_values(by=['vesselId', 'time'], inplace=True)

    # Feature Engineering
    train['prev_lat'] = train.groupby('vesselId')['latitude'].shift(n_shifts)
    train['prev_lon'] = train.groupby('vesselId')['longitude'].shift(n_shifts)
    train['prev_speed'] = train.groupby('vesselId')['sog'].shift(n_shifts)
    train['prev_course'] = (train.groupby('vesselId')['cog'].shift(n_shifts) / 180) - 1        # normalized
    train['prev_rotation'] = train.groupby('vesselId')['rot'].shift(n_shifts) 
    train['prev_heading'] = (train.groupby('vesselId')['heading'].shift(n_shifts)/ 180) - 1 
    train['hour'] = train['time'].dt.hour
    # Could change this to is_weekend
    train['day_of_week'] = train['time'].dt.dayofweek
    # Adding timedelta as a feature
    train['time_diff'] = train['time'].diff(n_shifts)
    train['time_diff_seconds'] = train['time_diff'].dt.total_seconds()

    # --------------------------------- prev_rot-related stuff
    # Replace special values with NaN
    train['prev_rotation'] = train['prev_rotation'].replace({127: np.nan, -127: np.nan, -128: np.nan})
    # Optional: Create a new column for turn information availability
    train['turn_info_available'] = np.where(train['prev_rotation'] == -128, 0, 1)

    # Create binary columns for turn direction and magnitude
    train['turn_direction'] = np.where(train['prev_rotation'] > 0, 'right', 'left')
    train['turn_magnitude'] = train['prev_rotation'].abs()

    # Fill missing values (optional, using forward fill) Uses most recent non-null value from the row above.
    train['prev_rotation'].fillna(method='ffill', inplace=True)

    # Drop rows with missing values
    train.dropna(inplace=True)

    print(f"Length of dataset after preprocessing: {len(train)}")
    return train
    


In [3]:

train1 = make_training_set(1)
train2 = make_training_set(2)
train3 = make_training_set(3)
train4 = make_training_set(4)
train5 = make_training_set(5)
train6 = make_training_set(6)
train7 = make_training_set(7)

train = pd.concat([train1, train2, train3, train4, train5, train6, train7], ignore_index=True)

C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['prev_rotation'].fillna(method='ffill', inplace=True)
C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['prev_rotation'].fillna(method='ffill', inplace=True)


Length of dataset after preprocessing: 893318


C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['prev_rotation'].fillna(method='ffill', inplace=True)
C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['prev_rotation'].fillna(method='ffill', inplace=True)


Length of dataset after preprocessing: 892648


C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['prev_rotation'].fillna(method='ffill', inplace=True)
C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['prev_rotation'].fillna(method='ffill', inplace=True)


...

Length of dataset after preprocessing: 890631


C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['prev_rotation'].fillna(method='ffill', inplace=True)
C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['prev_rotation'].fillna(method='ffill', inplace=True)


Length of dataset after preprocessing: 889969


C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['prev_rotation'].fillna(method='ffill', inplace=True)
C:\Users\mariu\AppData\Local\Temp\ipykernel_18720\2864780723.py:44: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  train['prev_rotation'].fillna(method='ffill', inplace=True)


Length of dataset after preprocessing: 889298


In [4]:
train['time_diff_seconds'].mean()

np.float64(30311.52442312245)

In [ ]:
feats_to_include = ['prev_lat', 'prev_lon', 'prev_speed', 'prev_course','prev_rotation', 'turn_info_available', 'prev_heading', 'time_diff_seconds']
X = train[feats_to_include]
y_lat = train['latitude']
y_lon = train['longitude']

X_lat_train, X_lat_val, y_lat_train, y_lat_val = train_test_split(X, y_lat, test_size=0.1, random_state=42)
X_lon_train, X_lon_val, y_lon_train, y_lon_val = train_test_split(X, y_lon, test_size=0.1, random_state=42)

# Train the model
model_lat = RandomForestRegressor(n_estimators=10, verbose=3, random_state=42)
model_lat.fit(X_lat_train.values, y_lat_train.values)

model_lon = RandomForestRegressor(n_estimators=10, verbose=3, random_state=42)
model_lon.fit(X_lon_train.values, y_lon_train.values)

# Make predictions on the validation set
y_lat_pred_val = model_lat.predict(X_lat_val)
y_lon_pred_val = model_lon.predict(X_lon_val)

# Evaluate performance on the validation set
mae_lat = mean_absolute_error(y_lat_val, y_lat_pred_val)
mae_lon = mean_absolute_error(y_lon_val, y_lon_pred_val)

print(f'Mean Absolute Error for Latitude: {mae_lat}')
print(f'Mean Absolute Error for Longitude: {mae_lon}')

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10


In [24]:
from datetime import datetime

filepath_train = '../datasets/ais_train.csv'

# Load AIS historical data
training_data = pd.read_csv(filepath_train, sep ='|')  # Replace with your dataset

# Predict future positions
def predict_future_position(id, vessel_id, time):
    # Fetch the latest known position of the vessel
    latest_data = training_data[training_data['vesselId'] == vessel_id].sort_values(by='time').iloc[-1]

    new_data = {
        'prev_lat': latest_data['latitude'],
        'prev_lon': latest_data['longitude'],
        'prev_speed': latest_data['sog'],
        'prev_course': (latest_data['cog'] / 180) - 1,
        'prev_rotation': latest_data['rot'],
        'turn_info_available': 1 if latest_data['rot'] != -128 else 0,
        'prev_heading' : (latest_data['heading'] / 180) - 1,

        # Convert the times to a datetime_obj
        'time_diff_seconds' : (datetime.strptime(time, '%Y-%m-%d %H:%M:%S') - datetime.strptime(latest_data['time'], '%Y-%m-%d %H:%M:%S')).total_seconds(),
    }
    # why is it done with all this list stuff?
    return id, model_lat.predict([list(new_data.values())])[0], model_lon.predict([list(new_data.values())])[0]
    # return id, model_lat.predict([list(new_data.values())]), model_lon.predict([list(new_data.values())]), new_data['time_diff_seconds'], latest_data['time']

# ['prev_lat', 'prev_lon', 'prev_speed', 'prev_course','prev_rotation', 'turn_info_available', 'prev_heading', 'time_diff_seconds']

In [25]:
from tqdm import tqdm
# Open the test file for reading and the prediction file for writing
with open('../datasets/ais_test.csv', 'r') as f_test, open('../predictions/predictions_2.csv', 'w') as f_pred:
    f_pred.write("ID,longitude_predicted,latitude_predicted\n")
    for line in f_test.readlines()[1:]:
        id, vesselID, time, scaling_factor = line.split(',')
        id, pred_lat, pred_lon = predict_future_position(id, vesselID, time)
        f_pred.write(f"{id},{pred_lon},{pred_lat}\n")